# Understanding the calculators in ASE

to wrap them in aiida-shell

In [1]:
import os
from koopmans.io import read
from koopmans.workflows import (DFTCPWorkflow, DFTPhWorkflow,
                                        KoopmansDFPTWorkflow,
                                        KoopmansDSCFWorkflow)

from koopmans.workflows import DFTPWWorkflow, WannierizeWorkflow

wf = read('ozone.json')

In [2]:
os.environ['PARA_PREFIX'] = 'mpirun -np 2'

In [3]:
workflow = KoopmansDFPTWorkflow.fromparent(wf)

In [4]:
workflow

KoopmansDFPTWorkflow(atoms=O3,
   parameters={'task': 'singlepoint', 'functional': 'ki', 'base_functional': 'pbe', ...},
   kpoints=Kpoints(gamma_only=False, grid=[1, 1, 1], offset=[0, 0, 0], path=G),
   pseudopotentials={'O': 'O_ONCV_PBE-1.2.upf'})

In [5]:
pw_workflow = DFTPWWorkflow.fromparent(workflow)

In [6]:
pw_params = pw_workflow.calculator_parameters['pw']
pw_params.nspin = 2
pw_params.tot_magnetization = 0

In [7]:
pw_calculator = pw_workflow.new_calculator('pw')

In [8]:
pw_calculator.command

mpirun -np 2 pw.x -in PREFIX.pwi > PREFIX.pwo 2>&1

In [9]:
pw_calculator.todict()

{'_parameters': {'calculation': 'scf', 'outdir': PosixPath('/home/jovyan/work/blitz/TMP'), 'prefix': 'kc', 'conv_thr': 3.6000000000000005e-08, 'verbosity': 'high', 'tot_charge': 0, 'tot_magnetization': 0, 'nbnd': 10, 'ecutwfc': 65.0, 'ecutrho': 260.0, 'kpts': [1, 1, 1], 'nspin': 2, 'pseudopotentials': {'O': 'O_ONCV_PBE-1.2.upf'}, 'pseudo_dir': PosixPath('/home/jovyan/project/blitz_shell_koopmans/codes/koopmans/src/koopmans/pseudopotentials/sg15_v1.2/pbe'), 'gamma_only': False, 'koffset': [0, 0, 0]},
 'atoms': Atoms(symbols='O3', pbc=False, cell=[14.1738, 12.0, 12.66], tags=..., calculator=PWCalculator(...)),
 'results': {},
 '_directory': PosixPath('/home/jovyan/work/blitz'),
 'prefix': 'espresso',
 'name': 'pwcalculator',
 'command': mpirun -np 2 pw.x -in PREFIX.pwi > PREFIX.pwo 2>&1,
 'calc': None,
 'skip_qc': False,
 '__koopmans_name__': 'PWCalculator',
 '__koopmans_module__': 'koopmans.calculators._pw'}

In [10]:
pw_calculator.calculate() # the error is expected, the mpirun is not installed in the current blitz env.

CalculationFailed: Calculator "pwcalculator" failed with command "mpirun -np 2 /home/jovyan/.aiida_venvs/blitz_shell_koopmans/bin/pw.x -in espresso.pwi > espresso.pwo 2>&1" failed in /home/jovyan/work/blitz with error code 127

In [11]:
pw_calculator.todict()

{'_parameters': {'calculation': 'scf', 'outdir': PosixPath('/home/jovyan/work/blitz/TMP'), 'prefix': 'kc', 'conv_thr': 3.6000000000000005e-08, 'verbosity': 'high', 'tot_charge': 0, 'tot_magnetization': 0, 'nbnd': 10, 'ecutwfc': 65.0, 'ecutrho': 260.0, 'kpts': [1, 1, 1], 'nspin': 2, 'pseudopotentials': {'O': 'O_ONCV_PBE-1.2.upf'}, 'pseudo_dir': PosixPath('/home/jovyan/project/blitz_shell_koopmans/codes/koopmans/src/koopmans/pseudopotentials/sg15_v1.2/pbe'), 'gamma_only': False, 'koffset': [0, 0, 0], 'ibrav': 0},
 'atoms': Atoms(symbols='O3', pbc=False, cell=[14.1738, 12.0, 12.66], tags=..., calculator=PWCalculator(...)),
 'results': {},
 '_directory': PosixPath('/home/jovyan/work/blitz'),
 'prefix': 'espresso',
 'name': 'pwcalculator',
 'command': 'mpirun -np 2 /home/jovyan/.aiida_venvs/blitz_shell_koopmans/bin/pw.x -in PREFIX.pwi > PREFIX.pwo 2>&1',
 'calc': None,
 'skip_qc': False,
 '__koopmans_name__': 'PWCalculator',
 '__koopmans_module__': 'koopmans.calculators._pw'}

### trying to use aiida-ase

non funziona, xk 1. é fatto per GPAW 2. richiede installato koopmans e tutto il resto sul cluster.

In [12]:
from aiida import common, engine, orm, plugins, load_profile
load_profile()

import copy

In [13]:
AseCalculation = plugins.CalculationFactory('ase.ase')

In [14]:
 # generate an example structure
structure = orm.StructureData(ase=pw_calculator.todict()['atoms'])

# k-point information
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([1,1,1])

In [15]:
# refine the dict, something cannot be serialized (like PosixPath)
refined_calculator = copy.deepcopy(pw_calculator.todict())

In [16]:
refined_calculator.pop('_directory',None)

PosixPath('/home/jovyan/work/blitz')

In [17]:
refined_calculator['_parameters'].pop('outdir')

PosixPath('/home/jovyan/work/blitz/TMP')

In [18]:
refined_calculator.pop('atoms')

Atoms(symbols='O3', pbc=False, cell=[14.1738, 12.0, 12.66], tags=..., calculator=PWCalculator(...))

In [19]:
refined_calculator['name'] = 'espresso'

In [20]:
refined_calculator

{'_parameters': {'calculation': 'scf', 'prefix': 'kc', 'conv_thr': 3.6000000000000005e-08, 'verbosity': 'high', 'tot_charge': 0, 'tot_magnetization': 0, 'nbnd': 10, 'ecutwfc': 65.0, 'ecutrho': 260.0, 'kpts': [1, 1, 1], 'nspin': 2, 'pseudopotentials': {'O': 'O_ONCV_PBE-1.2.upf'}, 'pseudo_dir': PosixPath('/home/jovyan/project/blitz_shell_koopmans/codes/koopmans/src/koopmans/pseudopotentials/sg15_v1.2/pbe'), 'gamma_only': False, 'koffset': [0, 0, 0], 'ibrav': 0},
 'results': {},
 'prefix': 'espresso',
 'name': 'espresso',
 'command': 'mpirun -np 2 /home/jovyan/.aiida_venvs/blitz_shell_koopmans/bin/pw.x -in PREFIX.pwi > PREFIX.pwo 2>&1',
 'calc': None,
 'skip_qc': False,
 '__koopmans_name__': 'PWCalculator',
 '__koopmans_module__': 'koopmans.calculators._pw'}

In [23]:
refined_calculator['_parameters'].pop('pseudo_dir')
refined_calculator['_parameters'].pop('pseudo_dir')

KeyError: 'pseudo_dir'

In [24]:
#refined_calculator['_parameters']['pseudo_dir'] = '/home/jovyan/project/blitz_shell_koopmans/codes/koopmans/src/koopmans/pseudopotentials/sg15_v1.2/pbe'

In [25]:
refined_calculator['_parameters']

{'calculation': 'scf', 'prefix': 'kc', 'conv_thr': 3.6000000000000005e-08, 'verbosity': 'high', 'tot_charge': 0, 'tot_magnetization': 0, 'nbnd': 10, 'ecutwfc': 65.0, 'ecutrho': 260.0, 'kpts': [1, 1, 1], 'nspin': 2, 'pseudopotentials': {'O': 'O_ONCV_PBE-1.2.upf'}, 'gamma_only': False, 'koffset': [0, 0, 0], 'ibrav': 0}

In [26]:
parameters = {
        'calculator': refined_calculator,
    }

settings = {'CMDLINE': ['python']}

In [27]:
builder = AseCalculation.get_builder()
builder.code = orm.load_code("ls@localhost")
builder.structure = structure
builder.kpoints = kpoints
builder.parameters = orm.Dict(parameters)
builder.settings = orm.Dict(settings)
builder.metadata.options.resources = {'num_machines': 1}
builder.metadata.options.max_wallclock_seconds = 30 * 60  # 30 minutes
builder.metadata.options.withmpi = False

In [28]:
from aiida.engine import run

In [29]:
run = run(builder)

03/21/2024 02:56:39 PM <3058784> aiida.engine.transports: [ERROR] Exception whilst using transport:
Traceback (most recent call last):
  File "/home/jovyan/project/blitz_shell_koopmans/codes/aiida-core/src/aiida/engine/processes/calcjobs/tasks.py", line 90, in do_upload
    calc_info = process.presubmit(folder)
  File "/home/jovyan/project/blitz_shell_koopmans/codes/aiida-core/src/aiida/engine/processes/calcjobs/calcjob.py", line 829, in presubmit
    calc_info = self.prepare_for_submission(folder)
  File "/home/jovyan/project/blitz_shell_koopmans/codes/aiida-ase/src/aiida_ase/calculations/ase.py", line 200, in prepare_for_submission
    if 'PW' in calc_args['mode'].values():
UnboundLocalError: local variable 'calc_args' referenced before assignment

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/jovyan/project/blitz_shell_koopmans/codes/aiida-core/src/aiida/engine/transports.py", line 105, in request_transport
   

PreSubmitException: exception occurred in presubmit call

## Trying aiida-shell

1. from ase calculator to aiida-shell inputs

The first thing to do is to write the input file.

In [30]:
pw_calculator.todict()

{'_parameters': {'calculation': 'scf', 'outdir': PosixPath('/home/jovyan/work/blitz/TMP'), 'prefix': 'kc', 'conv_thr': 3.6000000000000005e-08, 'verbosity': 'high', 'tot_charge': 0, 'tot_magnetization': 0, 'nbnd': 10, 'ecutwfc': 65.0, 'ecutrho': 260.0, 'kpts': [1, 1, 1], 'nspin': 2, 'pseudopotentials': {'O': 'O_ONCV_PBE-1.2.upf'}, 'pseudo_dir': PosixPath('/home/jovyan/project/blitz_shell_koopmans/codes/koopmans/src/koopmans/pseudopotentials/sg15_v1.2/pbe'), 'gamma_only': False, 'koffset': [0, 0, 0], 'ibrav': 0},
 'atoms': Atoms(symbols='O3', pbc=False, cell=[14.1738, 12.0, 12.66], tags=..., calculator=PWCalculator(...)),
 'results': {},
 '_directory': PosixPath('/home/jovyan/work/blitz'),
 'prefix': 'espresso',
 'name': 'pwcalculator',
 'command': 'mpirun -np 2 /home/jovyan/.aiida_venvs/blitz_shell_koopmans/bin/pw.x -in PREFIX.pwi > PREFIX.pwo 2>&1',
 'calc': None,
 'skip_qc': False,
 '__koopmans_name__': 'PWCalculator',
 '__koopmans_module__': 'koopmans.calculators._pw'}

In [31]:
ow_calculator.

NameError: name 'ow_calculator' is not defined